In [1]:
%pylab inline
import numpy as np
import nibabel as nib
from scipy.stats import pearsonr
from sklearn.decomposition import FastICA

Populating the interactive namespace from numpy and matplotlib


C:\Users\Lixuan\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def datafunction(img, atlas, region, n_components, filename):
    img = nib.load(img)
    atlas = nib.load(atlas)
    
    #turns data into matrix
    img_data = np.asarray(img._dataobj)
    atlas_data = np.asarray(atlas._dataobj)
    
    #Region refers to region number in ITK
    region_mask = (atlas_data == region) #extracts region data
    region_indices = np.nonzero(region_mask)
    
    ica = FastICA(n_components) 
    
    X = np.transpose(img_data[region_indices])
    icomps = ica.fit_transform(X)

    output = []
    for i, c in enumerate(icomps.T):
        output.append(np.zeros(X.shape[1]))
        for v in range (X.shape[1]):
            output[i][v] = pearsonr(c, X[:, v])[0]
    
    output_volume = np.zeros(atlas_data.shape)
    for i, (x, y, z) in enumerate(np.transpose(region_indices)):
        output_volume[x, y, z] = output[0][i]
        
    
    #Saves to file
    nii = nib.Nifti1Image(output_volume, atlas.affine)
    nii.to_filename(filename)
    print('Output saved to ' + filename)
    
    

In [3]:
Atlas = 'OneDrive_1_11-16-2018/Atlas/Rat_Paxinos_400um_bilat_180823_atlas.nii.gz'

imglist = ['OneDrive_1_11-16-2018/Data/sub-F01_task-rs_bold.nii.gz', 'OneDrive_1_11-16-2018/Data/sub-F02_task-rs_bold.nii.gz'
                   ,'OneDrive_1_11-16-2018/Data/sub-F03_task-rs_bold.nii.gz','OneDrive_1_11-16-2018/Data/sub-F04_task-rs_bold.nii.gz'
                   ,'OneDrive_1_11-16-2018/Data/sub-F05_task-rs_bold.nii.gz','OneDrive_1_11-16-2018/Data/sub-F06_task-rs_bold.nii.gz'
                   ,'OneDrive_1_11-16-2018/Data/sub-F07_task-rs_bold.nii.gz','OneDrive_1_11-16-2018/Data/sub-F08_task-rs_bold.nii.gz'
                   ,'OneDrive_1_11-16-2018/Data/sub-F09_task-rs_bold.nii.gz','OneDrive_1_11-16-2018/Data/sub-F10_task-rs_bold.nii.gz']
filename = ['1_comp.nii.gz', '2_comp.nii.gz', '3_comp.nii.gz', '4_comp.nii.gz', '5_comp.nii.gz', '6_comp.nii.gz',
            '7_comp.nii.gz', '8_comp.nii.gz', '9_comp.nii.gz', '10_comp.nii.gz']

In [4]:
for i in range(10):
    datafunction(imglist[i], Atlas, 27, 5, filename[i])



C:\Users\Lixuan\Anaconda3\lib\site-packages\sklearn\decomposition\fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


Output saved to 1_comp.nii.gz
Output saved to 2_comp.nii.gz
Output saved to 3_comp.nii.gz
Output saved to 4_comp.nii.gz
Output saved to 5_comp.nii.gz
Output saved to 6_comp.nii.gz
Output saved to 7_comp.nii.gz
Output saved to 8_comp.nii.gz
Output saved to 9_comp.nii.gz
Output saved to 10_comp.nii.gz


In [2]:
echo "# fMRI-processing-function" >> README.md
git init
git add README.md
git commit -m "first commit"
git remote add origin https://github.com/LeekSwan/fMRI-processing-function.git
git push -u origin master


SyntaxError: invalid syntax (<ipython-input-2-5323d5cbc1e7>, line 1)